# Real-world time series data
## Convolutions
This week we apply a convolution to get a better fitting model.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Try to learn 32 filters, 1 dimensional convolution with 5-values filter
    # We got rid of lambda layer that will shape the input for us, so we have to actually specify an `input_shape` in Conv1D
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="casual", activation="relu", input_shape=[None,1]),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 200)
])

optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)

model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
model.fit(dataset, epochs=500)

# We will have to update our `windowed_dataset` function we used before:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1) # expand dimensions of series before we process it

    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))

    return ds.batch(batch_size).prefetch(1)


After training we get a huge improvement over earlier results. The peak has lost its plato, but still is not quite right: it is not getting high enough relative to the data.
One solution might be to train a little bit longer as MAE and Loss are slowly decreasing.
We can also utilize LSTM layers:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="casual", activation="relu", input_shape=[None,1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 200)
])

Unfortunately it is overfitting and MAE goes down.
There is a lot of noise and instability. The one common cause of spikes is a [small batch size](https://www.youtube.com/watch?v=4qJaSmvhxi8) introducing more random noise. So it is worth experimenting with different batch_sizes.
